# Submit Jobrun

## 1. Input Data

In [6]:
import requests
import json
from io import BytesIO
import pandas as pd
from zipfile import ZipFile

job_data = pd.read_csv(r"C:\_Working\PAT_20201019\debug\test_jobrun_input2.csv") #data/test_jobrun.csv')
job_data

,PseudoPolicyID,PolicyID,Limit,Participation,Retention,PolPrem,Stack,TIV,RatingGroup
0,146975_4418990,146975,500000000,1.0,59462.42,513499.00,4418990,5467663.0,14.0
1,150448_4349144,150448,200000000,1.0,23782.53,369032.00,4349144,21565393.0,14.0
2,157768_4404398,157768,295895093,1.0,50273.97,253952.00,4404398,2856727.0,14.0
3,143094_4443112,143094,128762896,1.0,7890.00,96908.00,4443112,7890.0,14.0
4,140586_4423882,140586,13500000,1.0,14041.10,47985.00,4423882,3100000.0,39.0
...,...,...,...,...,...,...,...,...,...
188356,161645_4342004,161645,340554825,1.0,100000.00,1959921.78,4342004,955000.0,11.0
188357,143271_4387082,143271,117507191,1.0,17491.37,162241.00,4387082,34650321.0,11.0
188358,143219_4319949,143219,250000000,1.0,51804.44,231663.00,4319949,1515284.0,11.0
188359,156824_4404656,156824,100000000,1.0,502997.93,806999.00,4404656,24424723.0,39.0


## 2. Input Parameters

In [3]:
job_para = json.load(open('data/test_jobrun.json'))
print(json.dumps(job_para,indent=2))

{
  "data_source_type": "User_Upload",
  "valid_rules": 1073741826,
  "type_of_rating": "PSOLD",
  "curve_id": 2016,
  "psold": {
    "curve_persp": "Gross",
    "peril_subline": "All_Perils",
    "trend_factor": 1.035,
    "average_accident_date": "1/1/2022",
    "blending_type": "missing_invalid",
    "blending_weights": [
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0
    ],
    "hpr_blending": false
  },
  "loss_alae_ratio": 1,
  "additional_coverage": 2,
  "deductible_treatment": "Retains_Limit",
  "coverage_type": "Building_Contents_BI",
  "user_name": "Chengyou Xiao",
  "user_email": "chengyou.xiao@guycarp.com"
}


## 3. Request to the service

In [5]:
files = {
    'para': json.dumps(job_para),
    'data': job_data.to_csv(index=False)
}
resp = requests.post("http://frkcat-ws10:8002/api/job?jobrun=true", files=files)
if resp.status_code == 200:
    with ZipFile(BytesIO(resp.content),'r') as zf:
        df_res =  pd.read_csv(zf.open('pat_res.csv'))

df_res
# .sort_values('PseudoPolicyID')

# 20K records: 6s
# 188K: 28s
# 1m35s vs 26.3s

,PolicyID,Limit,Retention,PolPrem,LossRatio,participate,TIV,Stack,RatingGroup,LocID,AOIr,PolLAS,DedLAS,Premium
0,1,1000000,0,500000,0.8,0.2,8000000,990.0,1.0,1,37,29698.412590,0.000000,2.211518e+05
1,1,1000000,0,500000,0.8,0.2,9000000,0.0,7.0,20,37,37446.437912,0.000000,2.788482e+05
2,2,1000000,1000000,800000,0.7,0.4,4000000,990.0,2.0,2,34,40747.650608,35035.474479,1.924691e+05
3,2,1000000,1000000,800000,0.7,0.4,24000000,0.0,6.0,19,41,61494.150208,43463.598899,6.075309e+05
4,3,3000000,2000000,1000000,0.6,0.6,12000000,990.0,3.0,3,38,51849.015830,41883.543732,7.133598e+05
5,3,3000000,2000000,1000000,0.6,0.6,2600000,991.0,0.0,18,32,35809.195594,31804.898830,2.866402e+05
6,4,5000000,5000000,1500000,0.7,0.8,295000,990.0,4.0,4,22,13738.241730,13738.241730,0.000000e+00
7,4,5000000,5000000,1500000,0.7,0.8,17000000,991.0,0.0,17,40,50985.710521,45886.108142,1.500000e+06
8,5,500000,1000000,250000,0.8,0.0,1110000,990.0,5.0,5,28,27207.982167,25580.019324,8.041439e+04
9,5,500000,1000000,250000,0.8,0.0,27500000,991.0,0.0,16,42,36124.630787,32691.426228,1.695856e+05


In [4]:
resp.status_code, resp.headers, df_res.shape


(200,
 {'date': 'Fri, 21 Jan 2022 21:16:38 GMT', 'server': 'uvicorn', 'content-disposition': 'attachment;filename=pat_res.zip', 'access-control-expose-headers': 'Content-Disposition', 'content-type': 'application/zip', 'content-encoding': 'gzip', 'vary': 'Accept-Encoding', 'Transfer-Encoding': 'chunked'},
 (188361, 17))